<a href="https://colab.research.google.com/github/denny-erikson/dalilas-code-invoice/blob/main/Dalila's_Code_Invoice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dalila's Code Invoice 📜

In [ ]:
# @markdown ---

# @title Passo 1 - Preparação 🛠️
# @markdown 🔗 **Montando Diretórios:** Configurando o ambiente para acesso aos diretórios 📂
# @markdown - 🔒 **Solicitando Permissão de Acesso** ao Google Drive para carregar e salvar arquivos 🔐
# @markdown - ✅ **Aguarde** a execução completa desta etapa antes de prosseguir ⏳
# @markdown ---
# @markdown 🚨 **Importante:** Para garantir a integridade do processamento, **não abra nem edite este código!** ⚠️
# @markdown ---

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Passo 2 - Ambiente ⚡
# @markdown ⚡ **Preparação do embiente de execução:**
# @markdown - ✅ **Aguarde** a execução completa desta etapa ⏳

# @markdown ---
# @markdown ### 🛠 Instalações Necessárias:
# @markdown - **Poppler**: Utilizado para manipulação de arquivos PDF.
# @markdown - **Tesseract OCR (Português)**: Instalado para realizar reconhecimento óptico de caracteres (OCR) em português.
# @markdown - **Python Libraries**:
# @markdown   - `pytesseract` e `Pillow`: Para realizar operações de OCR e manipulação de imagem.
# @markdown   - `pdf2image`: Para converter páginas PDF em imagens.
# @markdown   - `pandas` e `openpyxl`: Para manipulação e exportação de dados em formato Excel.

# @markdown ---
# @markdown 🚨 **Importante:** Para garantir a integridade do processamento, **não abra nem edite este código!** ⚠️
# @markdown ---
!apt-get clean
!apt-get autoremove

!sudo apt-get update
!apt-get install -y poppler-utils # Instala o Poppler
!apt-get install tesseract-ocr-por # Installs the Portuguese language data for Tesseract
!pip install pytesseract Pillow pdf2image # Keep your existing installations
!pip install pandas openpyxl

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Pack

In [ ]:
# @title Passo 3 - Carregar Fornecedores ⚡
# @markdown ⚡ **Descrição da Etapa:** Nesta etapa, carregamos o arquivo Excel contendo os dados dos fornecedores.
# @markdown - ✅ **Aguarde** a execução completa desta etapa ⏳
# @markdown - O arquivo **FORNECEDORES CORRIGIDO.xlsx** é lido, e selecionamos apenas as colunas relevantes para o processamento, especificamente:
# @markdown   - **CNPJ/CPF**: Identificação única do fornecedor
# @markdown   - **Nome**: Nome do fornecedor

# @markdown ---
# @markdown 🚨 **Importante:** Para garantir a integridade do processamento, **não abra nem edite este código!** ⚠️
# @markdown ---


import pandas as pd

# Caminho do arquivo Excel
caminho_excel = "/content/drive/Shareddrives/dalilas_code_homologacao/FORNECEDORES CORRIGIDO.xlsx"
# Carregar a planilha do Excel e selecionar apenas as colunas desejadas
df = pd.read_excel(caminho_excel, usecols=['CNPJ/CPF', 'Nome'])

In [ ]:
# @title Passo 4 - Filtros ⚡
# @markdown ⚡ **Descrição da Etapa:** Nesta etapa, aplicamos várias funções de filtro e processamento para extrair informações específicas do texto e dos dados carregados:
# @markdown - ✅ **Aguarde** a execução completa desta etapa ⏳
# @markdown - **carregar_e_filtrar_excel**: Carrega o arquivo Excel de fornecedores e filtra pelo CNPJ/CPF, removendo zeros à esquerda e formatações (pontos, barras e hífens).
# @markdown - **buscar_cnpj**: Busca o CNPJ no texto, verificando se ele não está na lista de CNPJs ignorados.
# @markdown - **encontrar_data**: Extrai uma data no formato DD/MM/AAAA do texto e a converte para o formato desejado, com o mês escrito por extenso.
# @markdown - **identificar_filial**: Identifica a filial correspondente ao CNPJ encontrado no texto, com base em um dicionário de mapeamento.
# @markdown - **validar_numero_nota_fiscal**: Procura o número da nota fiscal no texto, usando uma lista de expressões regulares para cobrir diferentes formatos possíveis.

# @markdown ---
# @markdown 🚨 **Importante:** Para garantir a integridade do processamento, **não abra nem edite este código!** ⚠️
# @markdown ---




import re

# Função para carregar o Excel, extrair as colunas e filtrar pelo CNPJ/CPF
def carregar_e_filtrar_excel(df, cnpj_cpf_filtro):
    regex_formatacao = r'[./-]'

    # Remove formatações do CNPJ/CPF
    cleaned_format = re.sub(regex_formatacao, '', cnpj_cpf_filtro)

    # Regex para remover zeros à esquerda, preservando ao menos um dígito
    regex_zeros = r'^0+'

    # Remove zeros à esquerda
    cnpj_removed_left_zeros = re.sub(regex_zeros, '', cleaned_format)

    # Filtra o DataFrame pelo CNPJ/CPF fornecido
    resultado = df[df['CNPJ/CPF'].astype(str) == str(cnpj_removed_left_zeros)]

    # Verifica se o filtro resultou em algum dado
    if not resultado.empty:
        print("Nome encontrado:", resultado['Nome'].values[0])
        return resultado['Nome'].values[0]
    else:
        print("CNPJ/CPF não encontrado.")
        return "CNPJ/CPF não encontrado"


# Função que busca o CNPJ no texto e compara com uma lista de CNPJs a serem ignorados
def buscar_cnpj(texto):
    # Lista de CNPJs que serão ignorados
    cnpjs_ignorados = ['62.438.841/0001-32', '62.438.841/0004-85', '62.438.841/0006-47']

    # Regex para capturar CNPJ no formato xx.xxx.xxx/xxxx-xx
    regex_cnpj = r'\b\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}\b'

    # Procura o CNPJ no texto
    match = re.search(regex_cnpj, texto)

    if match:
        cnpj_encontrado = match.group(0)

        # Verifica se o CNPJ não está na lista de CNPJs ignorados
        if cnpj_encontrado not in cnpjs_ignorados:
            print("CNPJ encontrado:", cnpj_encontrado)
            return cnpj_encontrado
        else:
            print(f"CNPJ encontrado está na lista de ignorados: {cnpj_encontrado}")
            return None  # Não carregar se for um CNPJ da lista de ignorados
    else:
        print("Nenhum CNPJ encontrado no texto.")
        return None


# Exemplo de como usar as funções
def processar_informacoes(df, texto):
    # Chama a função buscar_cnpj e aguarda o resultado
    cnpj_cpf_filtro = buscar_cnpj(texto)

    # Só chama carregar_e_filtrar_excel se um CNPJ/CPF válido for encontrado
    if cnpj_cpf_filtro:
        name = carregar_e_filtrar_excel(df, cnpj_cpf_filtro)
        return name
    else:
        print("Nenhum CNPJ válido para buscar no Excel.")
        return "CNPJ não encontrado ou é o mesmo CNPJ específico."


def encontrar_data(texto):
    # Regex para capturar datas no formato DD/MM/YYYY
    regex_data = r'\b\d{2}/\d{2}/\d{4}\b'

    # Procura a data no texto
    match = re.search(regex_data, texto)

    if match:
        # Retorna a data encontrada
        return match.group(0)
    else:
        return "Nenhuma data encontrada."


def identificar_filial(texto):
    # Dicionário que mapeia CNPJs às filiais correspondentes
    cnpj_filiais = {
        '62.438.841/0001-32': "NR SAM",
        '62.438.841/0006-47': "NR SAP",
        '62.438.841/0004-85': "NR SP"
    }

    # Regex para capturar CNPJ no formato xx.xxx.xxx/xxxx-xx
    regex_cnpj = r'\b\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}\b'

    # Procura todos os CNPJs no texto
    cnpjs_encontrados = re.findall(regex_cnpj, texto)

    # Lista para armazenar as filiais encontradas
    filiais_encontradas = []

    # Verifica cada CNPJ encontrado e retorna a filial correspondente
    for cnpj in cnpjs_encontrados:
        if cnpj in cnpj_filiais:
            filiais_encontradas.append(cnpj_filiais[cnpj])

    # Retorna as filiais encontradas ou uma mensagem caso não encontre nenhuma
    if filiais_encontradas:
        return filiais_encontradas
    else:
        return "Nenhum CNPJ correspondente encontrado."


def encontrar_data(texto):
    # Regex para capturar datas no formato DD/MM/YYYY
    regex_data = r'\b\d{2}/\d{2}/\d{4}\b'

    # Procura a data no texto
    match = re.search(regex_data, texto)

    if match:
        # Obtém a data encontrada
        data_encontrada = match.group(0)

        # Divide a data em componentes
        dia, mes, ano = data_encontrada.split('/')

        # Dicionário para mapear o número do mês para o nome do mês
        meses = {
            '01': 'JANEIRO', '02': 'FEVEREIRO', '03': 'MARÇO',
            '04': 'ABRIL', '05': 'MAIO', '06': 'JUNHO',
            '07': 'JULHO', '08': 'AGOSTO', '09': 'SETEMBRO',
            '10': 'OUTUBRO', '11': 'NOVEMBRO', '12': 'DEZEMBRO'
        }

        # Obtem o nome do mês
        nome_mes = meses[mes]

        # Retorna a data no formato desejado
        return f"{ano}/{nome_mes}/{dia}"
    else:
        return "Nenhuma data encontrada."


def validar_numero_nota_fiscal(texto):
    # Lista de expressões regulares para capturar o número da nota fiscal
    first_case_regex_list = [
        r'Número\s*(\d+)',
        r'NÚMERO\s*(\d+)',
        r'Número da Nota\s*(\d+)',
        r'Número da NFS-e\s*(\d+)',
        r'Nº\s*(\d+)',
        r'Nota Fiscal\s*:\s*(\d+)',
        r'NF\s*:\s*(\d+)',
        r'Nota\s*:\s*(\d+)',
        r'Fatura\s*:\s*(\d+)',
        r'Número da NFS-e Competência da NFS-e\s*:\s*(\d+)',
        r'Número / Série\s*:\s*(\d+)',
        r'Número \s*/\s* Série\s*:\s*(\d+)',
        r'NFS-e\s*:\s*(\d+)',
        r' NFS-e\s*:\s*(\d+)',
        r'- NFS-e\s*:\s*(\d+)',
        r' \s*-\s* \s*NFS-e\s*:\s*(\d+)',
        r'Número da\n NFS-e\n\s*:\s*(\d+)',
        r'Número \s*da\n NFS-e\n\s*:\s*(\d+)',
        r'\s*-\s*NFS-e\s*(\d+)',



    ]

    second_case_regex_list = [
      r'(?i)Número (da NFS-e|da Nota)?\s*(\d+)',
      r'(?i)NFS-e\s*:\s*(\d+)',
      r'(?i)NF\s*:\s*(\d+)',
      r'(?i)Nota Fiscal\s*:\s*(\d+)',
      r'(?i)Nota\s*:\s*(\d+)',
      r'(?i)Fatura\s*:\s*(\d+)',
      r'(?i)Nº\s*(\d+)',
      r'(?i)Número\s*/\s*Série\s*:\s*(\d+)',
      r'\s*-\s*NFS-e\s*:\s*(\d+)',
      r'\s*-\s*NFS-e\s*(\d+)',
  ]


    # Itera pelas expressões regulares e procura por correspondências

    for regex_list in [first_case_regex_list]:
      for regex in regex_list:
          match = re.search(regex, texto)
          if match:
              return match.group(1)  # Retorna o primeiro número encontrado

    return "Número da nota fiscal não encontrado."  # Retorno se nenhuma correspondência for encontrada





In [ ]:
# @title Passo 5 - Processamento ⚡
# @markdown ⚡ **Descrição da Etapa:** Neste passo, implementamos a função `extract_info_and_save`, que realiza o seguinte:
# @markdown - **Verificação de Extensão de Arquivo**: A função começa verificando a extensão do arquivo de entrada para determinar se ele é um PDF.
# @markdown - **Inicialização de Métricas**: Um dicionário chamado `metrics` é criado para armazenar informações relevantes, como data de execução, tempo de execução, filial, data, número da NFS-e, nome/razão social, CNPJ e o caminho do arquivo salvo.
# @markdown - **Extração de Informações**: A função `extract_info` é responsável por extrair informações como o número da NFS-e, nome/razão social e CNPJ a partir do texto extraído das imagens do PDF. Essa função utiliza expressões regulares para localizar e validar as informações.
# @markdown - **Conversão de PDF para Imagens**: Se o arquivo for um PDF, ele é convertido em imagens utilizando a biblioteca `pdf2image`. O Tesseract é então utilizado para extrair texto de cada página.
# @markdown - **Salvamento de Arquivos**: Após a extração das informações, cada página do PDF é salva como um novo arquivo PDF. O caminho do arquivo salvo é registrado nas métricas.
# @markdown - **Registro de Erros**: Qualquer erro encontrado durante o processamento é registrado em um arquivo de log.
# @markdown - **Armazenamento de Métricas**: As métricas de execução são salvas em um arquivo CSV, incluindo informações sobre a execução do script, como tempo e data.
# @markdown - **Notificação de Conclusão**: Ao final do processamento, uma mensagem de alerta é exibida para indicar que o script foi concluído com sucesso.

# @markdown Este processo é crucial para a automação da extração de dados de documentos fiscais, facilitando a organização e o acesso às informações necessárias.
# @markdown ---
# @markdown 🚨 **Importante:** Para garantir a integridade do processamento, **não abra nem edite este código!** ⚠️
# @markdown ---



import pytesseract
from PIL import Image
import re
import os
from pdf2image import convert_from_path
from datetime import datetime
from IPython.display import Javascript
import time
from datetime import timedelta
import csv  # Para salvar as métricas em CSV

# Função para exibir um alerta ao término do script
def alert_message(message):
    display(Javascript(f'alert("{message}")'))

# Define the TESSDATA_PREFIX environment variable
os.environ['TESSDATA_PREFIX'] = '/usr/share/tesseract-ocr/4.00/tessdata'  # Set the path for tessdata

def extract_info_and_save(file_path, save_dir):
    # Verifica a extensão do arquivo
    file_extension = os.path.splitext(file_path)[1].lower()
    # Variável para armazenar o log de erros
    error_log = []

    # Dicionário para armazenar métricas
    metrics = {
        "Data de Execução": None,
        "Tempo de Execução": None,
        "Filial": None,
        "Data": None,
        "Número NFS-e": None,
        "Nome/Razão Social": None,
        "CNPJ": None,
        "Caminho Arquivo Salvo": None
    }

    start_time = time.time()  # Inicia o cronômetro

    # Função para encontrar as informações de "Número da NFS-e" e "Nome/Razão Social"
    def extract_info(text):
        nfs_e_number = validar_numero_nota_fiscal(text)

        filiais = identificar_filial(text)
        print("Filial encontrada:", filiais)

        # Armazenando a filial encontrada
        metrics["Filial"] = filiais  # Armazena a filial no dicionário

        data_formatada = encontrar_data(text)
        print("Data encontrada:", data_formatada)

        # Armazenando a data encontrada
        metrics["Data"] = data_formatada

        numero_encontrado = validar_numero_nota_fiscal(text)
        print("Número da Nota Fiscal encontrado:", numero_encontrado)

        # Armazenando o número NFS-e encontrado
        metrics["Número NFS-e"] = numero_encontrado

        try:
            # Aqui verificamos se a função buscar_cnpj e carregar_e_filtrar_excel retornam valores válidos
            name = processar_informacoes(df, text)
            metrics["Nome/Razão Social"] = name  # Armazena o nome no dicionário
            if not isinstance(name, str):
                raise ValueError("Nome retornado não é uma string.")
        except Exception as e:
            # # Expressão regular para capturar o "Nome/Razão Social"
            name_match = re.search(r'Nome/Razão Social\s*(.+)', text)
            # # Verifica se encontrou um nome, se não tenta carregar e filtrar pelo Excel
            if name_match:
                name = name_match.group(1).strip()
                metrics["Nome/Razão Social"] = name  # Armazena o nome no dicionário

        # Aqui você pode adicionar a lógica para capturar o CNPJ, se necessário
        cnpj_match = re.search(r'\b\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}\b', text)  # Exemplo de regex para CNPJ
        if cnpj_match:
            metrics["CNPJ"] = cnpj_match.group(0)  # Armazena o CNPJ no dicionário

        return nfs_e_number, name

    def save_text_as_pdf(text, save_path):
        # Salvando texto completo em um arquivo .txt
        text_file_path = os.path.splitext(save_path)[0] + '.txt'
        with open(text_file_path, 'w', encoding='utf-8') as text_file:
            text_file.write(text)
        print(f"Texto completo salvo como: {text_file_path}")

    if file_extension == '.pdf':
        # Converte PDF para imagens
        images = convert_from_path(file_path)

        # Usa o Tesseract para extrair texto de cada página do PDF
        for page_number, image in enumerate(images, start=1):
            text = pytesseract.image_to_string(image, lang='por')  # Extrai texto de cada página

            # Extrai as informações
            try:
                nfs_e_number, name = extract_info(text)

                # Salva cada página como um PDF separado
                safe_name = re.sub(r'[<>:"/\\|?*]', '', name)  # Remove caracteres inválidos para nomes de arquivos
                safe_nfs_e_number = nfs_e_number if nfs_e_number != "Não encontrado" else "0000"  # Valor padrão
                formatted_folder_name = f"NFSe-{safe_nfs_e_number}-{safe_name}-Page-{page_number}"
                formatted_file_name = f"NFSe-{safe_nfs_e_number}-{safe_name}-Page-{page_number}{file_extension}"

                # Define o caminho completo para salvar
                save_directory = os.path.join(save_dir, formatted_folder_name)
                os.makedirs(save_directory, exist_ok=True)

                save_path = os.path.join(save_directory, formatted_file_name)  # Caminho completo para salvar o arquivo

                # Salva a imagem da página como PDF
                image.save(save_path, "PDF")
                print(f"Arquivo PDF da página {page_number} salvo como: {save_path}")

                save_text_as_pdf(text, save_path)

                # Armazenando o caminho do arquivo salvo
                metrics["Caminho Arquivo Salvo"] = save_path

            except Exception as e:
                error_log.append(f"Falha na leitura/salvamento da página {page_number}: {str(e)}")

        os.remove(file_path)  # Remove o arquivo PDF original

        # Calcula o tempo de execução
        end_time = time.time()
        execution_time_seconds = end_time - start_time
        metrics["Tempo de Execução"] = str(timedelta(seconds=execution_time_seconds))
        metrics["Data de Execução"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Armazena a data de execução

    # Se houver erros, registra no log
    if error_log:
        log_file_path = os.path.join(save_dir, "error_log.txt")
        with open(log_file_path, "w") as log_file:
            for error in error_log:
                log_file.write(error + "\n")
        print(f"Erros registrados em: {log_file_path}")

    # Caminho do arquivo CSV
    metrics_file_path = "/content/drive/Shareddrives/dalilas_code_homologacao/data/dalilas-metrics.csv"
    # Verifica se o arquivo já existe
    file_exists = os.path.isfile(metrics_file_path)
    # Abre o arquivo em modo de adição
    with open(metrics_file_path, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        # Se o arquivo não existir, escreve os cabeçalhos
        if not file_exists:
            writer.writerow(metrics.keys())  # Escreve os cabeçalhos
        writer.writerow(metrics.values())  # Escreve os valores
    print(f"Métricas salvas em: {metrics_file_path}")

    alert_message("Script finalizado com sucesso!")


In [ ]:
# @title Dalila's Code Invoice 📜
# @markdown #### 🗂️ *Selecione a Pasta de Destino para Salvamento*
# @markdown ##### 🔄 Etapas para garantir o sucesso do processamento:
# @markdown - ✅ **Verifique** se a pasta de destino existe no Google Drive ⏳
# @markdown - ✅ **Preencha** os campos corretamente para garantir a integridade do processamento ✏️
# @markdown - ✅ **Carregue** a imagem ou PDF que deseja processar 📤
# @markdown - ✅ **Aguarde** a execução da função `extract_info_and_save` 📋
# @markdown - ✅ **Processo finalizado com sucesso!** 🎉
# @markdown - 📂 Para acessar os resultados, vá até a pasta especificada: *Notas Fiscais Entrada/Nota Fiscal Serviço* 🗂️
# @markdown - 📊 **Dica**: Confira os logs para garantir que tudo está conforme esperado.


# @markdown - 👊🏽 **Obrigado por utilizar o Dalila's Code** 🚀
# @markdown ---



import os
shared_or_drive_path = "/content/drive/Shareddrives/Financeiro" # @param ["/content/drive/Shareddrives/dalilas_code_homologacao/Financeiro","/content/drive/MyDrive/NFSe","/content/drive/Shareddrives/Financeiro"]
branch_path = "NR SP" # @param ["NR SAM","NR SAP", "NR SP"]
year_path = "2025" # @param ["2024","2025"]
month_path = "FEVEREIRO" # @param ["JANEIRO", "FEVEREIRO", "MARÇO", "ABRIL", "MAIO", "JUNHO", "JULHO", "AGOSTO", "SETEMBRO", "OUTUBRO", "NOVEMBRO", "DEZEMBRO"]
day_path = "28" # @param ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31"]

if(os.path.exists(shared_or_drive_path)):
  print(shared_or_drive_path)

  # Carregar a imagem ou PDF
  from google.colab import files
  uploaded = files.upload()

  # Define a pasta onde os arquivos serão salvos (exemplo: '/content/drive/MyDrive/NFSe')
  base_directory = f"{shared_or_drive_path}/Notas Fiscais Entrada/Nota Fiscal Serviço/{branch_path}/{year_path}/{month_path}/{day_path}"

  save_directory = os.path.join(base_directory)

  # Executar a função
  file_path = list(uploaded.keys())[0]  # Obtém o nome do arquivo carregado
  extract_info_and_save(file_path, save_directory)

else:
    print("Pasta não encontrada")


/content/drive/Shareddrives/Financeiro


Saving Scan2025-03-13_084313.pdf to Scan2025-03-13_084313.pdf
Filial encontrada: ['NR SP']
Data encontrada: 2025/FEVEREIRO/24
Número da Nota Fiscal encontrado: 3797737
CNPJ encontrado está na lista de ignorados: 62.438.841/0004-85
Nenhum CNPJ válido para buscar no Excel.
Arquivo PDF da página 1 salvo como: /content/drive/Shareddrives/Financeiro/Notas Fiscais Entrada/Nota Fiscal Serviço/NR SP/2025/FEVEREIRO/28/NFSe-3797737-CNPJ não encontrado ou é o mesmo CNPJ específico.-Page-1/NFSe-3797737-CNPJ não encontrado ou é o mesmo CNPJ específico.-Page-1.pdf
Texto completo salvo como: /content/drive/Shareddrives/Financeiro/Notas Fiscais Entrada/Nota Fiscal Serviço/NR SP/2025/FEVEREIRO/28/NFSe-3797737-CNPJ não encontrado ou é o mesmo CNPJ específico.-Page-1/NFSe-3797737-CNPJ não encontrado ou é o mesmo CNPJ específico.-Page-1.txt
Filial encontrada: ['NR SP']
Data encontrada: 2025/FEVEREIRO/07
Número da Nota Fiscal encontrado: 195167
CNPJ encontrado: 19.184.315/0001-64
Nome encontrado: SURVEYM

<IPython.core.display.Javascript object>